In [ ]:
## delimiters

import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from prettyprinter import pprint
import os
import re
import shutil

# file = "C:\\Users\\Lenovo\\Desktop\\Oaknorth\\working\\xebia_1_page_3.jpg"
# k = 1
for file in glob.glob('C:\\air_ticket\\images2\\*'):
#     print(file)



# filename = img_name+'_delimiters.jpg'
# file = 'C:\\air_ticket\\preprocessed2\\Yatra_AAAIN233676564_26.09.2022_page0.jpg'
# file = 'C:\\Users\\Lenovo\\Desktop\\Oaknorth\\working\\amplexor_1_page_5.jpg'
    filename = os.path.split(file)[-1].split('.jpg')[0]
    # img_name = os.path.split(file)[-1].split('.')[0]

    table_image_contour = cv2.imread(file, 0)
    table_image = cv2.imread(file)

    ret, thresh_value = cv2.threshold(
        table_image_contour, 180, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5,5),np.uint8)
    dilated_value = cv2.dilate(thresh_value,kernel,iterations = 1)

    contours, hierarchy = cv2.findContours(
        dilated_value, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    coords = []
    new_contours = []
    # max_height = -1

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # print(w*h)
        # bounding the images
        # if h > max_height:
        #     max_height = h
        if w*h > 150:
            new_contours.append(cnt)
            coords.append([x,y,x+w,y+h])
            table_image = cv2.rectangle(table_image, (x, y), (x + w, y + h), (0, 0, 0), 1)
    cv2.imwrite('samples\\{}.jpg'.format(filename), table_image)
    # k += 1
    # print(k)

    # Calculate maximum rectangle height
    c = np.array(contours)
    # co = list(c)
    # pprint(co)

    # max_height = np.max(c[::, 3])

    max_height = 10
    # Sort the contours by y-value
    by_y = sorted(coords, key=lambda x: x[1])  # y values

    line_y = by_y[0][1]       # first y
    line = 1
    by_line = []

    # Assign a line number to each contour
    for x, y, w, h in by_y:
        if y > line_y + max_height:
            line_y = y
            line += 1
            
        by_line.append([line, x, y, w, h])
    # pprint(by_line)

    img = cv2.imread(file)

    # This will now sort automatically by line then by x
    contours_sorted = [[line, x, y, w, h] for line, x, y, w, h in sorted(by_line)]
    # pprint(contours_sorted)
    # for x, y, w, h in contours_sorted:
        # print(x,y,x+w,y+h)
        # print(f"{x:4} {y:4} {w:4} {h:4}")
    start = 0
    # column_names = ['date description']


    # pprint(contours_sorted)
    # 32, 504, 734, 561

    # roi = img[483:521, 31:2360]
    # roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    # cv2.imwrite('roi.jpg', roi)
    # cv2.imshow('roi', roi)
    # cv2.waitKey()

    grouped_contours = []
    subgroup = []

    subgroup.append(contours_sorted[0])

    for i in range(len(contours_sorted)-1):
        if contours_sorted[i][0] == contours_sorted[i+1][0]:
            subgroup.append(contours_sorted[i])
        else:
            subgroup.append(contours_sorted[i])
            grouped_contours.append(subgroup)
            subgroup = []

    # pprint(grouped_contours)

    for subgroup in grouped_contours:

        ## error handling if -+5 coordinates goes out of scope of image
        try:
            x1 = subgroup[0][1]-5
            y1 = subgroup[0][2]-5
            x2 = subgroup[-1][3]+5
            y2 = subgroup[-1][4]+5

            roi = img[y1:y2, x1:x2]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            cv2.imwrite('roi.jpg', roi)
        
        except:
            x1 = subgroup[0][1]
            y1 = subgroup[0][2]
            x2 = subgroup[-1][3]
            y2 = subgroup[-1][4]

            roi = img[y1:y2, x1:x2]
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            cv2.imwrite('roi.jpg', roi)

        command = 'tesseract roi.jpg output -l eng --psm 4'
        os.system(command)
        text = ''
        with open('output.txt', encoding='utf-8') as f:
            text = f.read()
        # print(text.lower())
        # print(re.search('.*date.*balance.*', text.lower()))
        if re.search('.*pax.*sector.*', text.lower()):
            line = subgroup[0][0]
            # print(text)
            break

    # pprint(contours_sorted)

    flag = 0
    req = []

    for i in range(start, len(contours_sorted)-1):
        if contours_sorted[i][0] >= line:
            if flag == 0:
                print(contours_sorted[i])
            flag = 1

            # print(contours_sorted[i+1][0]-contours_sorted[i][2])
        if flag == 1:
            if contours_sorted[i+1][1] - contours_sorted[i][3] > 10 and contours_sorted[i][0] == contours_sorted[i+1][0]:
                # cv2.putText(img, "|", ((contours_sorted[i][3]+contours_sorted[i+1][1])//2, (contours_sorted[i][2]+contours_sorted[i][4])//2), cv2.FONT_HERSHEY_SIMPLEX,  
                                            # 1, (0, 0, 0), 1, cv2.LINE_AA)
                img = cv2.line(img, ((contours_sorted[i][3]+contours_sorted[i+1][1])//2, contours_sorted[i][2]-3), ((contours_sorted[i][3]+contours_sorted[i+1][1])//2, contours_sorted[i][4]+3), (0,0,0), 2)
                cv2.imwrite('delimiters\\'+filename+'.jpg', img)

    # img = cv2.imread('sample.jpg')
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # cv2.imwrite('sample.jpg', img)

    # command = 'tesseract sample.jpg stdout -l eng --psm 4'
    # print(os.popen(command).read())
    # pprint(req)

    delimiter_image = 'delimiters\\'+filename+'.jpg'
    preproc_image = 'preprocessed_delimiters\\'+filename+'.jpg'
    shutil.copy(delimiter_image, preproc_image)

    combinations = ['set_dpi-convert_to_rgb-remove_noise']

    for combination in combinations:
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(preproc_image, img)

        parsed_output = 'preprocessed_delimiters\\'+filename+'_'+combination
        command = 'tesseract {} {} -l eng --psm 4'.format(preproc_image, parsed_output)
        os.system(command)

In [ ]:
## add delimiters with character coordinates

## delimiters

import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from prettyprinter import pprint
import os
import re
import shutil

# file = "C:\\Users\\Lenovo\\Desktop\\Oaknorth\\working\\xebia_1_page_3.jpg"
# k = 1
for file in glob.glob('C:\\air_ticket\\images2\\*'):
    filename = os.path.split(file)[-1].split('.jpg')[0]

    output_file = 'box_files\\new'
    
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('new.jpg', img)

    filename = 'new.jpg'

    command = 'tesseract {} {} batch.nochop makebox'.format(filename, output_file)
    os.system(command)

    box_file = 'box_files\\new.box'

    text = ''
    with open(box_file, 'r', encoding='utf-8') as fp:
        text = fp.read()

    text = text.splitlines()
    # pprint(text)

    ch_coords = []

    for ch in text:
        ch_split = ch.split()
        ch_coords.append(ch_split)

    # pprint(ch_coords)

    lines = []
    line = []

    for i in range(len(ch_coords)-1):
        c = ch_coords[i][0]

        if int(ch_coords[i][1]) <= int(ch_coords[i+1][1]):
            line.append(ch_coords[i])
        else:
            line.append(ch_coords[i])
            lines.append(line)
            line = []

    delimiter_image = 'delimiters\\'+filename+'.jpg'
    preproc_image = 'preprocessed_delimiters\\'+filename+'.jpg'
    shutil.copy(delimiter_image, preproc_image)

    combinations = ['set_dpi-convert_to_rgb-remove_noise']

    for combination in combinations:
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(preproc_image, img)

        parsed_output = 'preprocessed_delimiters\\'+filename+'_'+combination
        command = 'tesseract {} {} -l eng --psm 4'.format(preproc_image, parsed_output)
        os.system(command)

In [32]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    im = Image.open(image)
    im.save(preproc_image, dpi=(600,600))
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def deskew(image):
    im = io.imread(image)
    angle = determine_skew(im)
    rotated = rotate(im, angle, resize=True) * 255
    io.imsave(image, rotated.astype(np.uint8))
    return image

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('images2\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

# combinations = ['set_dpi-convert_to_grayscale', 'set_dpi-increase_contrast-convert_to_grayscale', 'set_dpi-convert_to_grayscale-remove_noise', 'set_dpi-increase_contrast-convert_to_grayscale-remove_noise', 'set_dpi-convert_to_grayscale-deskew', 'set_dpi-increase_contrast-convert_to_grayscale-deskew', 'set_dpi-convert_to_grayscale-remove_noise-deskew', 'set_dpi-increase_contrast-convert_to_grayscale-remove_noise-deskew']
combinations = ['set_dpi-convert_to_rgb-remove_noise']

for image in glob.glob('sample.jpg'):
    filename = os.path.split(image)[-1].split('.jpg')[0]
    preproc_image = 'sample2.jpg'
    shutil.copy(image, preproc_image)
    for combination in combinations:
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(preproc_image, img)

        parsed_output = filename+'_'+combination
        command = 'tesseract {} {} -l eng --psm 4'.format(preproc_image, parsed_output)
        os.system(command)
